In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip3 install pypianoroll

In [2]:
import pypianoroll as ppr
import numpy as np
import torch
import torch.nn as nn
import pickle

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [44]:
noise = torch.randn(1, 1024, device=device)

In [8]:
class MultiResBlock(nn.Module):
    def __init__(self,ch_in,ch_out):
        super().__init__()
        self.conv1x1 = nn.Conv2d(ch_in, ch_out, kernel_size=1, padding=0)
        self.bnorm1x1 = nn.BatchNorm2d(ch_out,track_running_stats=False)
        self.fconv = nn.Conv2d(ch_in, ch_out//6, kernel_size=3, padding=1)
        self.fbnorm = nn.BatchNorm2d(ch_out//6,track_running_stats=False)
        self.sconv = nn.Conv2d(ch_out//6, ch_out//3, kernel_size=3, padding=1)
        self.sbnorm = nn.BatchNorm2d(ch_out//3,track_running_stats=False)
        self.tconv = nn.Conv2d(ch_out//3, ch_out//2+1, kernel_size=3, padding=1)
        self.tbnorm = nn.BatchNorm2d(ch_out//2+1,track_running_stats=False)
        self.relu = nn.ReLU()

    def forward(self,x):
        res1x1 = self.relu(self.bnorm1x1(self.conv1x1(x)))
        #print("res1x1 done")
        first = self.relu(self.fbnorm(self.fconv(x)))
        #print("fconv done")
        second = self.relu(self.sbnorm(self.sconv(first)))
        third = self.relu(self.tbnorm(self.tconv(second)))
        resconv = torch.cat((first,second,third),dim=1)
        y = res1x1+resconv
        return y

In [9]:
class generator(nn.Module):    
    def __init__(self):
        super().__init__()
        '''
        Conv2d: w' = (w - k +2p)/s + 1
        ConvTr2d: w = s(w'-1) + k -2p
        '''
        #[b, 512, 4, 4]
        self.tr1 = nn.ConvTranspose2d(512, 384, kernel_size = 4, stride = 2, padding = 1) # [b, 384, 8, 8]
        self.tr2 = nn.ConvTranspose2d(384, 256, kernel_size = 3, stride = 1, padding = 1) # [b, 256, 8, 8]
        self.tr3 = nn.ConvTranspose2d(256, 192, kernel_size = 4, padding = 1, stride = 2) # [b, 192, 16, 16]
        self.tr4 = nn.ConvTranspose2d(192, 128, kernel_size = 3, stride = 1, padding = 1) # [b, 128, 16, 16]
        self.tr5 = nn.ConvTranspose2d(128, 96, kernel_size = 4, padding = 1, stride = 2) # [b, 96, 32, 32]
        self.tr6 = nn.ConvTranspose2d(96, 64, kernel_size = 3, stride = 1, padding = 1) # [b, 64, 32, 32]
        self.tr7 = nn.ConvTranspose2d(64, 48, kernel_size = 4, padding = 1, stride = 2) # [b, 48, 32, 32]
        self.tr8 = nn.ConvTranspose2d(48, 32, kernel_size = 3, stride = 1, padding = 1) # [b, 32, 64, 64]
        self.tr9 = nn.ConvTranspose2d(32, 16, kernel_size = 4, padding = 1, stride = 2) # [b, 16, 64, 64]
        self.tr10 = nn.ConvTranspose2d(16, 8, kernel_size = 3, stride = 1, padding = 1) # [b, 8, 128, 128]
        self.tr11 = nn.ConvTranspose2d(8, 4, kernel_size = 3, padding = 1, stride = 1) # [b, 4, 128, 128]
        self.tr12 = nn.ConvTranspose2d(4, 1, kernel_size = 3, stride = 1, padding = 1) # [b, 1, 128, 128]
        
        
        self.bn1 = nn.BatchNorm2d(384)
        self.bn2 = nn.BatchNorm2d(256)
        self.bn3 = nn.BatchNorm2d(192)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(96)
        self.bn6 = nn.BatchNorm2d(64)
        self.bn7 = nn.BatchNorm2d(48)
        self.bn8 = nn.BatchNorm2d(32)
        self.bn9 = nn.BatchNorm2d(16)
        self.bn10 = nn.BatchNorm2d(8)
        self.bn11 = nn.BatchNorm2d(4)
        
        '''
        self.tr1 = nn.ConvTranspose2d(256, 128, kernel_size = 4, stride = 2, padding = 1)
        self.tr2 = nn.ConvTranspose2d(128, 64, kernel_size = 4, stride = 2, padding = 1)
        self.tr3 = nn.ConvTranspose2d(64, 32, kernel_size = 4, stride = 2, padding = 1)
        self.tr4 = nn.ConvTranspose2d(32, 16, kernel_size = 3, stride = 1, padding = 1)
        self.tr5 = nn.ConvTranspose2d(16, 8, kernel_size = 3, stride = 1, padding = 1)
        self.tr6 = nn.ConvTranspose2d(8, 1, kernel_size = 3, stride = 1, padding = 1)'''
        self.relu = nn.ReLU()
        self.fc = nn.Linear(1024, 512*4*4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 512, 4, 4)
        z1 = self.bn1(self.relu(self.tr1(z)))
        # z = [b, 512, 4, 4], z1 = [b, 384, 8, 8], z2 [b, 256, 8, 8] -> [b, 640, 8, 8]
        z2 = self.bn2(self.relu(self.tr2(z1)))
        z3 = self.bn3(self.relu(self.tr3(z2)))
        z4 = self.bn4(self.relu(self.tr4(z3)))
        z5 = self.bn5(self.relu(self.tr5(z4)))
        z6 = self.bn6(self.relu(self.tr6(z5)))
        z7 = self.bn7(self.relu(self.tr7(z6)))
        z8 = self.bn8(self.relu(self.tr8(z7)))
        z9 = self.bn9(self.relu(self.tr9(z8)))
        z10 = self.bn10(self.relu(self.tr10(z9)))
        z11 = self.bn11(self.relu(self.tr11(z10)))
        z12 = self.sigmoid(self.tr12(z11))
        return z12

In [10]:
class Generator_Res(nn.Module):
    def __init__(self):
        super().__init__()
        self.tr1 = MultiResBlock(512, 256) 
        self.tr2 = MultiResBlock(256, 128)
        self.tr3 = MultiResBlock(128, 64)
        self.tr4 = MultiResBlock(64, 32)
        self.tr5 = MultiResBlock(32, 16)
        self.tr6 = MultiResBlock(16, 8)
        self.tr7 = nn.Conv2d(8, 1, kernel_size = 3, stride = 1, padding = 1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(1024, 512*4*4)
        self.sigmoid = nn.Sigmoid()
        self.upsample = nn.Upsample(scale_factor = 2)
        
        self.up1 = nn.ConvTranspose2d(256, 256, kernel_size = 4, stride = 2, padding = 1)
        self.up2 = nn.ConvTranspose2d(128, 128, kernel_size = 4, stride = 2, padding = 1)
        self.up3 = nn.ConvTranspose2d(64, 64, kernel_size = 4, stride = 2, padding = 1)
        self.up4 = nn.ConvTranspose2d(32, 32, kernel_size = 4, stride = 2, padding = 1)
        self.up5 = nn.ConvTranspose2d(16, 16, kernel_size = 4, stride = 2, padding = 1)
        


    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 512, 4, 4)
        z = self.tr1(z)
        z = self.up1(z) # 8x8
        z = self.tr2(z)
        z = self.up2(z) # 16x16
        z = self.tr3(z)
        z = self.up3(z) # 32x32
        z = self.tr4(z)
        z = self.up4(z) # 64x64
        z = self.tr5(z)
        z = self.up5(z) # 128x128
        z = self.tr6(z)
        z = self.tr7(z)
        z = self.sigmoid(z)
        return z

In [11]:
class Generator_Res2(nn.Module):
    def __init__(self):
        super().__init__()
        self.tr1 = MultiResBlock(256, 128)
        self.tr2 = MultiResBlock(128, 64)
        self.tr3 = MultiResBlock(64, 32)
        self.tr4 = MultiResBlock(32, 16)
        self.tr5 = MultiResBlock(16, 8)
        self.tr6 = nn.Conv2d(8, 1, kernel_size = 3, stride = 1, padding = 1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(1024, 256*16*16)
        self.sigmoid = nn.Sigmoid()
        self.upsample = nn.Upsample(scale_factor = 2)

    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 256, 16, 16)
        z = self.tr1(z)
        z = self.upsample(z)
        z = self.tr2(z)
        z = self.upsample(z)
        z = self.tr3(z)
        z = self.upsample(z)
        z = self.tr4(z)
        z = self.tr5(z)
        z = self.tr6(z)
        z = self.sigmoid(z)
        return z

In [13]:
generator = generator().to(device)
generator.load_state_dict(torch.load('Generator_normal_10epochs.pt'))

<All keys matched successfully>

In [45]:
generator.eval()
fake = generator(noise)
fake_cpu = fake.detach().cpu().numpy()

In [46]:
fake_cpu = fake_cpu.reshape((128,128))*127

In [3]:
with open('lstm_gan.pickle', 'rb') as test_pickle:
    fake_cpu = pickle.load(test_pickle)

In [5]:
fake_cpu = fake_cpu.detach().cpu().numpy()

In [7]:
fake_cpu.shape

(1, 128, 128)

In [8]:
track = ppr.Track(fake_cpu[0])
multi = ppr.Multitrack(tracks=[track], beat_resolution=8)

In [42]:
multi

Multitrack(tracks=['unknown'], tempo=array([120.]), downbeat=None, beat_resolution=8, name=unknown)

In [9]:
ppr.write(multi, 'gan.mid')

In [ ]:
fake_cpu